In [1]:
import pulp
import pandas as pd
import numpy as np

In [2]:
# Define the problem
prob = pulp.LpProblem("Incoming_Bus_Allocation", pulp.LpMinimize)

In [3]:
# Define sets
solo_routes = ['ALABANG','BINAN','CARMONA','BALIBAGO','CABUYAO','CALAMBA']  # List of solo routes
hybrid_routes = [('ALABANG','CARMONA'),('BINAN','CARMONA'),('CALAMBA','CABUYAO')]  # List of hybrid routes (each element is a tuple (i, j))

# Import parameters (to be replaced with actual data import code)
cost_small_bus = {}  # Dictionary with costs for small buses on solo routes
cost_large_bus = {}  # Dictionary with costs for large buses on solo routes
cost_small_hybrid_route = {}  # Dictionary with costs for small hybrid routes
cost_large_hybrid_route = {}  # Dictionary with costs for large hybrid routes
capacity_small_bus = 18
capacity_large_bus = 56
buffer_current_small = {}  # Dictionary with buffer capacities for small buses
buffer_current_large = {}  # Dictionary with buffer capacities for large buses
demand = {}  # Dictionary of demands for each route

In [4]:
# Loading cost rates from the spreadsheet
rates_file_path = 'Bus Rate cleaned.xlsx'
xls = pd.ExcelFile(rates_file_path)

# Load the sheet into a DataFrame
cost_df = pd.read_excel(xls, sheet_name=0)

# Load the sheet into a DataFrame
cost_df = pd.read_excel(xls, sheet_name=0)

# Extracting costs
for index, row in cost_df.iterrows():
    route = row['ROUTE']
    total_large = row['TOTAL']
    total_small = row['TOTAL.1']
    
    # Check if it's a hybrid route
    if 'VIA' in route:
        route_parts = route.split(' VIA ')
        route_tuple = (route_parts[1], route_parts[0])
        cost_large_hybrid_route[route_tuple] = total_large
        cost_small_hybrid_route[route_tuple] = total_small
    else:
        cost_large_bus[route] = total_large
        cost_small_bus[route] = total_small

In [5]:
# Add bus capacities and buffer size

#Edit this buffer if needed. For now, it's a flat 3 people buffer per bus
buffer_small = 0
buffer_large = 3

for r in solo_routes:
    buffer_current_small[r]=buffer_small
    buffer_current_large[r]=buffer_large
    
for hr in hybrid_routes:
    buffer_current_small[hr]=buffer_small
    buffer_current_large[hr]=buffer_large

In [6]:
#Demands 

# Load the data
demand_file_path = 'cleaned_bus_shuttle_data_v31.csv'
bus_data = pd.read_csv(demand_file_path)

# Create a dictionary of dictionaries to store the 'ROUTE':'TOTAL' for each shift for each day
shift_data = {}

# Iterate through each unique date
for date in bus_data['DATE'].unique():
    shift_data[date] = {}
    date_data = bus_data[bus_data['DATE'] == date]
    
    # Iterate through each unique shift
    for shift in date_data['SHIFT'].unique():
        shift_data[date][shift] = {}
        shift_data_date_shift = date_data[date_data['SHIFT'] == shift]
        
        # Populate the dictionary for each route and its total
        for _, row in shift_data_date_shift.iterrows():
            route_name = row['ROUTE'].upper()
            shift_data[date][shift][route_name] = row['TOTAL']

# EDIT THIS IF NECESSARY
# Select one of the dictionaries for testing
test_date = '25'  # Select the date
test_shift = 'OUT 4PM'  # Select the shift
test_dict = shift_data[test_date][test_shift]

# Display the selected dictionary for testing
# test_dict

demand = test_dict

In [7]:
demand

{'ALABANG': 4,
 'BINAN': 6,
 'CARMONA': 4,
 'BALIBAGO': 31,
 'TAGAPO': 0,
 'CABUYAO': 4,
 'CALAMBA': 7}

In [8]:
# Preprocessing to exclude NaN values
def preprocess_costs(cost_dict):
    return {k: v for k, v in cost_dict.items() if not pd.isna(v)}

cost_small_bus = preprocess_costs(cost_small_bus)
cost_large_bus = preprocess_costs(cost_large_bus)
cost_small_hybrid_route = preprocess_costs(cost_small_hybrid_route)
cost_large_hybrid_route = preprocess_costs(cost_large_hybrid_route)

# Updated sets based on preprocessed costs
solo_routes = [route for route in solo_routes if route in cost_small_bus or route in cost_large_bus]
hybrid_routes = [route for route in hybrid_routes if route in cost_small_hybrid_route or route in cost_large_hybrid_route]

print("Preprocessed solo routes:", solo_routes)
print("Preprocessed hybrid routes:", hybrid_routes)
print("Preprocessed cost_small_bus:", cost_small_bus)
print("Preprocessed cost_large_bus:", cost_large_bus)
print("Preprocessed cost_small_hybrid_route:", cost_small_hybrid_route)
print("Preprocessed cost_large_hybrid_route:", cost_large_hybrid_route)

Preprocessed solo routes: ['ALABANG', 'BINAN', 'CARMONA', 'BALIBAGO', 'CABUYAO', 'CALAMBA']
Preprocessed hybrid routes: [('ALABANG', 'CARMONA'), ('BINAN', 'CARMONA'), ('CALAMBA', 'CABUYAO')]
Preprocessed cost_small_bus: {'ALABANG': 1423.5, 'BINAN': 586.5, 'BALIBAGO': 1038.0, 'CABUYAO': 586.5, 'CALAMBA': 1300.5}
Preprocessed cost_large_bus: {'ALABANG': 3072, 'BINAN': 2460, 'CARMONA': 1881, 'BALIBAGO': 1881, 'CABUYAO': 1935, 'CALAMBA': 2127}
Preprocessed cost_small_hybrid_route: {('ALABANG', 'CARMONA'): 1518.0, ('BINAN', 'CARMONA'): 630.0, ('CALAMBA', 'CABUYAO'): 630.0}
Preprocessed cost_large_hybrid_route: {('ALABANG', 'CARMONA'): 3270, ('BINAN', 'CARMONA'): 2550, ('CALAMBA', 'CABUYAO'): 2625}


# Define decision variables (assuming these are already defined in your original code)
x_small = pulp.LpVariable.dicts("x_small", solo_routes, lowBound=0, cat='Integer')
x_large = pulp.LpVariable.dicts("x_large", solo_routes, lowBound=0, cat='Integer')
y_small = pulp.LpVariable.dicts("y_small", hybrid_routes, lowBound=0, cat='Integer')
y_large = pulp.LpVariable.dicts("y_large", hybrid_routes, lowBound=0, cat='Integer')

# Objective Function
prob += (
    pulp.lpSum([cost_small_bus[i] * x_small[i] for i in solo_routes if i in cost_small_bus]) +
    pulp.lpSum([cost_large_bus[i] * x_large[i] for i in solo_routes if i in cost_large_bus]) +
    pulp.lpSum([cost_small_hybrid_route[(i, j)] * y_small[(i, j)] for (i, j) in hybrid_routes if (i, j) in cost_small_hybrid_route]) +
    pulp.lpSum([cost_large_hybrid_route[(i, j)] * y_large[(i, j)] for (i, j) in hybrid_routes if (i, j) in cost_large_hybrid_route])
)

# Define decision variables
x_small = pulp.LpVariable.dicts("x_small", solo_routes, lowBound=0, cat='Integer')
x_large = pulp.LpVariable.dicts("x_large", solo_routes, lowBound=0, cat='Integer')
y_small = pulp.LpVariable.dicts("y_small", hybrid_routes, lowBound=0, cat='Integer')
y_large = pulp.LpVariable.dicts("y_large", hybrid_routes, lowBound=0, cat='Integer')

# Objective function
prob += (
    pulp.lpSum([cost_small_bus[i] * x_small[i] for i in solo_routes if i in cost_small_bus]) +
    pulp.lpSum([cost_large_bus[i] * x_large[i] for i in solo_routes if i in cost_large_bus]) +
    pulp.lpSum([cost_small_hybrid_route[i_j] * y_small[i_j] for i_j in hybrid_routes if i_j in cost_small_hybrid_route]) +
    pulp.lpSum([cost_large_hybrid_route[i_j] * y_large[i_j] for i_j in hybrid_routes if i_j in cost_large_hybrid_route])
)

# Constraints
# Demand Satisfaction for Solo and Hybrid Routes
for i in solo_routes:
    prob += (
        (capacity_small_bus * x_small[i] if i in cost_small_bus else 0) +
        (capacity_large_bus * x_large[i] if i in cost_large_bus else 0) +
        pulp.lpSum([(capacity_small_bus - buffer_current_small[i_j]) * y_small[i_j] if i_j in cost_small_hybrid_route else 0 for i_j in hybrid_routes if i in i_j]) +
        pulp.lpSum([(capacity_large_bus - buffer_current_large[i_j]) * y_large[i_j] if i_j in cost_large_hybrid_route else 0 for i_j in hybrid_routes if i in i_j]) -
        pulp.lpSum([(capacity_small_bus - buffer_current_small[k_i]) * y_small[k_i] if k_i in cost_small_hybrid_route else 0 for k_i in hybrid_routes if i in k_i]) -
        pulp.lpSum([(capacity_large_bus - buffer_current_large[k_i]) * y_large[k_i] if k_i in cost_large_hybrid_route else 0 for k_i in hybrid_routes if i in k_i])
        >= demand.get(i, 0)
    )

# Handling Combined Demand for Hybrid Routes
for (i, j) in hybrid_routes:
    prob += (
        (capacity_small_bus - buffer_current_small[(i, j)]) * y_small[(i, j)] if (i, j) in cost_small_hybrid_route else 0 +
        (capacity_large_bus - buffer_current_large[(i, j)]) * y_large[(i, j)] if (i, j) in cost_large_hybrid_route else 0
        >= demand.get(i, 0) + demand.get(j, 0)
    )

# Handling Excess Demand for Route i
for i in solo_routes:
    prob += (
        (x_small[i] * capacity_small_bus if i in cost_small_bus else 0) +
        (x_large[i] * capacity_large_bus if i in cost_large_bus else 0) >= 
        demand.get(i, 0) - pulp.lpSum([(capacity_small_bus - buffer_current_small[i_j]) * y_small[i_j] if i_j in cost_small_hybrid_route else 0 for i_j in hybrid_routes if i in i_j]) -
        pulp.lpSum([(capacity_large_bus - buffer_current_large[i_j]) * y_large[i_j] if i_j in cost_large_hybrid_route else 0 for i_j in hybrid_routes if i in i_j])
    )

# Handling Excess Demand for Route j
for j in solo_routes:
    prob += (
        (x_small[j] * capacity_small_bus if j in cost_small_bus else 0) +
        (x_large[j] * capacity_large_bus if j in cost_large_bus else 0) >= 
        demand.get(j, 0) - pulp.lpSum([(capacity_small_bus - buffer_current_small[i_j]) * y_small[i_j] if i_j in cost_small_hybrid_route else 0 for i_j in hybrid_routes if j in i_j]) -
        pulp.lpSum([(capacity_large_bus - buffer_current_large[i_j]) * y_large[i_j] if i_j in cost_large_hybrid_route else 0 for i_j in hybrid_routes if j in i_j])
    )

# Cost Efficiency for Hybrid Routes
for i_j in hybrid_routes:
    prob += (
        (cost_small_hybrid_route[i_j] * y_small[i_j] if i_j in cost_small_hybrid_route else 0) +
        (cost_large_hybrid_route[i_j] * y_large[i_j] if i_j in cost_large_hybrid_route else 0) <= 
        pulp.lpSum([cost_small_bus[i] * y_small[i_j] if i in cost_small_bus else 0 + cost_large_bus[i] * y_large[i_j] if i in cost_large_bus else 0 for i in i_j])
    )

# Zero Demand Constraint
for i in solo_routes:
    if demand.get(i, 0) == 0:
        prob += x_small[i] == 0
        prob += x_large[i] == 0

for (i, j) in hybrid_routes:
    if demand.get(i, 0) == 0 and demand.get(j, 0) == 0:
        prob += y_small[(i, j)] == 0
        prob += y_large[(i, j)] == 0

# NaN Cost Constraint
for i in solo_routes:
    if i not in cost_small_bus:
        prob += x_small[i] == 0
    if i not in cost_large_bus:
        prob += x_large[i] == 0

for (i, j) in hybrid_routes:
    if (i, j) not in cost_small_hybrid_route:
        prob += y_small[(i, j)] == 0
    if (i, j) not in cost_large_hybrid_route:
        prob += y_large[(i, j)] == 0

# Solve the problem with integer constraints
prob.solve(pulp.PULP_CBC_CMD(msg=True))

# Output results
print("\nSolution:")
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Total Cost =", pulp.value(prob.objective))

# Define decision variables
x_small = pulp.LpVariable.dicts("x_small", solo_routes, lowBound=0, cat='Integer')
x_large = pulp.LpVariable.dicts("x_large", solo_routes, lowBound=0, cat='Integer')
y_small = pulp.LpVariable.dicts("y_small", hybrid_routes, lowBound=0, cat='Integer')
y_large = pulp.LpVariable.dicts("y_large", hybrid_routes, lowBound=0, cat='Integer')

# Objective function
prob += (
    pulp.lpSum([cost_small_bus[i] * x_small[i] for i in solo_routes if i in cost_small_bus]) +
    pulp.lpSum([cost_large_bus[i] * x_large[i] for i in solo_routes if i in cost_large_bus]) +
    pulp.lpSum([cost_small_hybrid_route[i_j] * y_small[i_j] for i_j in hybrid_routes if i_j in cost_small_hybrid_route]) +
    pulp.lpSum([cost_large_hybrid_route[i_j] * y_large[i_j] for i_j in hybrid_routes if i_j in cost_large_hybrid_route])
)

# Constraints
# Demand Satisfaction for Solo and Hybrid Routes
for i in solo_routes:
    prob += (
        (capacity_small_bus * x_small[i] if i in cost_small_bus else 0) +
        (capacity_large_bus * x_large[i] if i in cost_large_bus else 0) +
        pulp.lpSum([(capacity_small_bus - buffer_current_small[i_j]) * y_small[i_j] for i_j in hybrid_routes if i in i_j and i_j in cost_small_hybrid_route]) +
        pulp.lpSum([(capacity_large_bus - buffer_current_large[i_j]) * y_large[i_j] for i_j in hybrid_routes if i in i_j and i_j in cost_large_hybrid_route]) -
        pulp.lpSum([(capacity_small_bus - buffer_current_small[k_i]) * y_small[k_i] for k_i in hybrid_routes if i in k_i and k_i in cost_small_hybrid_route]) -
        pulp.lpSum([(capacity_large_bus - buffer_current_large[k_i]) * y_large[k_i] for k_i in hybrid_routes if i in k_i and k_i in cost_large_hybrid_route])
        >= demand.get(i, 0)
    )

# Handling Combined Demand for Hybrid Routes
for (i, j) in hybrid_routes:
    prob += (
        ((capacity_small_bus - buffer_current_small[(i, j)]) * y_small[(i, j)] if (i, j) in cost_small_hybrid_route else 0) +
        ((capacity_large_bus - buffer_current_large[(i, j)]) * y_large[(i, j)] if (i, j) in cost_large_hybrid_route else 0)
        >= demand.get(i, 0) + demand.get(j, 0)
    )

# Handling Excess Demand for Route i
for i in solo_routes:
    prob += (
        ((x_small[i] * capacity_small_bus) if i in cost_small_bus else 0) +
        ((x_large[i] * capacity_large_bus) if i in cost_large_bus else 0) >= 
        demand.get(i, 0) - pulp.lpSum([(capacity_small_bus - buffer_current_small[i_j]) * y_small[i_j] if i in i_j and i_j in cost_small_hybrid_route else 0 for i_j in hybrid_routes]) -
        pulp.lpSum([(capacity_large_bus - buffer_current_large[i_j]) * y_large[i_j] if i in i_j and i_j in cost_large_hybrid_route else 0 for i_j in hybrid_routes])
    )

# Handling Excess Demand for Route j
for j in solo_routes:
    prob += (
        ((x_small[j] * capacity_small_bus) if j in cost_small_bus else 0) +
        ((x_large[j] * capacity_large_bus) if j in cost_large_bus else 0) >= 
        demand.get(j, 0) - pulp.lpSum([(capacity_small_bus - buffer_current_small[i_j]) * y_small[i_j] if j in i_j and i_j in cost_small_hybrid_route else 0 for i_j in hybrid_routes]) -
        pulp.lpSum([(capacity_large_bus - buffer_current_large[i_j]) * y_large[i_j] if j in i_j and i_j in cost_large_hybrid_route else 0 for i_j in hybrid_routes])
    )

# Cost Efficiency for Hybrid Routes
for i_j in hybrid_routes:
    prob += (
        ((cost_small_hybrid_route[i_j] * y_small[i_j]) if i_j in cost_small_hybrid_route else 0) +
        ((cost_large_hybrid_route[i_j] * y_large[i_j]) if i_j in cost_large_hybrid_route else 0) <= 
        pulp.lpSum([cost_small_bus[i] * y_small[i_j] if i in cost_small_bus else 0 + cost_large_bus[i] * y_large[i_j] if i in cost_large_bus else 0 for i in i_j])
    )

# Zero Demand Constraint
for i in solo_routes:
    if demand.get(i, 0) == 0:
        prob += x_small[i] == 0
        prob += x_large[i] == 0

for (i, j) in hybrid_routes:
    if demand.get(i, 0) == 0 and demand.get(j, 0) == 0:
        prob += y_small[(i, j)] == 0
        prob += y_large[(i, j)] == 0

'''
# NaN Cost Constraint
for i in solo_routes:
    if i not in cost_small_bus:
        prob += x_small[i] == 0
    if i not in cost_large_bus:
        prob += x_large[i] == 0

for (i, j) in hybrid_routes:
    if (i, j) not in cost_small_hybrid_route:
        prob += y_small[(i, j)] == 0
    if (i, j) not in cost_large_hybrid_route:
        prob += y_large[(i, j)] == 0
'''

# Print the problem data
print("Problem data:")
print(prob)

# Solve the problem with integer constraints
prob.solve(pulp.PULP_CBC_CMD(msg=True))

# Output results
print("\nSolution:")
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Total Cost =", pulp.value(prob.objective))

# Output results
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Total Cost =", pulp.value(prob.objective))

# Decision Variables
x_small = pulp.LpVariable.dicts("x_small", solo_routes, lowBound=0, cat='Integer')
x_large = pulp.LpVariable.dicts("x_large", solo_routes, lowBound=0, cat='Integer')
y_small = pulp.LpVariable.dicts("y_small", hybrid_routes, lowBound=0, cat='Integer')
y_large = pulp.LpVariable.dicts("y_large", hybrid_routes, lowBound=0, cat='Integer')

# Objective Function
prob += (
    pulp.lpSum([cost_small_bus[i] * x_small[i] for i in solo_routes if cost_small_bus.get(i) is not None]) +
    pulp.lpSum([cost_large_bus[i] * x_large[i] for i in solo_routes if cost_large_bus.get(i) is not None]) +
    pulp.lpSum([cost_small_hybrid_route[(i, j)] * y_small[(i, j)] for (i, j) in hybrid_routes if cost_small_hybrid_route.get(i, j) is not None]) +
    pulp.lpSum([cost_large_hybrid_route[(i, j)] * y_large[(i, j)] for (i, j) in hybrid_routes if cost_large_hybrid_route.get(i, j) is not None])
)

# Constraints

# 1. Demand Satisfaction for Solo and Hybrid Routes
for i in solo_routes:
    prob += (
        ((capacity_small_bus - buffer_current_small.get(i, 0)) * x_small[i] if cost_small_bus.get(i) is not None else 0) +
        ((capacity_large_bus - buffer_current_large.get(i, 0))* x_large[i] if cost_large_bus.get(i) is not None else 0) +
        pulp.lpSum([capacity_y_small[i_j] - buffer_current_small.get(i_j, 0) + capacity_y_large[i_j] - buffer_current_small.get(i_j, 0) for i_j in hybrid_routes if i in i_j])
        >= demand.get(i, 0)
    )

# 2. Capacity Non-Negativity (implicitly handled by lowBound=0 in variable definitions)

# 3. Zero Demand Constraint
for i in solo_routes:
    if demand[i] == 0:
        prob += x_small[i] == 0
        prob += x_large[i] == 0

for (i, j) in hybrid_routes:
    if (demand[i] == 0) and (demand[j] == 0):
        prob += y_small[(i, j)] == 0
        prob += y_large[(i, j)] == 0

# Solve the problem
prob.solve()

# Output the results
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")

print(f"Total Cost = {pulp.value(prob.objective)}")

# Define the problem
prob = pulp.LpProblem("Incoming_Bus_Allocation2", pulp.LpMinimize)

# Decision Variables
x_small = pulp.LpVariable.dicts("x_small", solo_routes, lowBound=0, cat='Integer')
x_large = pulp.LpVariable.dicts("x_large", solo_routes, lowBound=0, cat='Integer')
y_small = pulp.LpVariable.dicts("y_small", hybrid_routes, lowBound=0, cat='Integer')
y_large = pulp.LpVariable.dicts("y_large", hybrid_routes, lowBound=0, cat='Integer')

# Objective Function
prob += (
    pulp.lpSum([cost_small_bus[i] * x_small[i] for i in solo_routes if i in cost_small_bus]) +
    pulp.lpSum([cost_large_bus[i] * x_large[i] for i in solo_routes if i in cost_large_bus]) +
    pulp.lpSum([cost_small_hybrid_route[(i, j)] * y_small[(i, j)] for (i, j) in hybrid_routes if (i, j) in cost_small_hybrid_route]) +
    pulp.lpSum([cost_large_hybrid_route[(i, j)] * y_large[(i, j)] for (i, j) in hybrid_routes if (i, j) in cost_large_hybrid_route])
)

# Constraints

# 1. Demand Satisfaction for Solo and Hybrid Routes
for i in solo_routes:
    prob += (
        pulp.lpSum([
            (capacity_small_bus - buffer_current_small.get(i, 0)) * x_small[i] if cost_small_bus.get(i) is not None else 0,
            (capacity_large_bus - buffer_current_large.get(i, 0)) * x_large[i] if cost_large_bus.get(i) is not None else 0
        ]) +
        pulp.lpSum([
            (capacity_small_bus - buffer_current_small.get((i, j), 0)) * y_small[(i, j)] if cost_small_hybrid_route.get((i, j)) is not None else 0
            for (i, j) in hybrid_routes if i in (i, j)
        ]) +
        pulp.lpSum([
            (capacity_large_bus - buffer_current_large.get((i, j), 0)) * y_large[(i, j)] if cost_large_hybrid_route.get((i, j)) is not None else 0
            for (i, j) in hybrid_routes if i in (i, j)
        ])
        >= demand.get(i, 0)
    )


# 2. Capacity Non-Negativity (implicitly handled by lowBound=0 in variable definitions)

# 3. Zero Demand Constraint
for i in solo_routes:
    if demand[i] == 0:
        prob += x_small[i] == 0
        prob += x_large[i] == 0

for (i, j) in hybrid_routes:
    if (demand[i] == 0) and (demand[j] == 0):
        prob += y_small[(i, j)] == 0
        prob += y_large[(i, j)] == 0

# Solve the problem
prob.solve()

# Output the results
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")

print(f"Total Cost = {pulp.value(prob.objective)}")


# Decision Variables
x_small = pulp.LpVariable.dicts("x_small", solo_routes, lowBound=0, cat='Integer')
x_large = pulp.LpVariable.dicts("x_large", solo_routes, lowBound=0, cat='Integer')
y_small = pulp.LpVariable.dicts("y_small", hybrid_routes, lowBound=0, cat='Integer')
y_large = pulp.LpVariable.dicts("y_large", hybrid_routes, lowBound=0, cat='Integer')

# Objective Function
prob += (
    pulp.lpSum([cost_small_bus[i] * x_small[i] for i in solo_routes if i in cost_small_bus]) +
    pulp.lpSum([cost_large_bus[i] * x_large[i] for i in solo_routes if i in cost_large_bus]) +
    pulp.lpSum([cost_small_hybrid_route[(i, j)] * y_small[(i, j)] for (i, j) in hybrid_routes if (i, j) in cost_small_hybrid_route]) +
    pulp.lpSum([cost_large_hybrid_route[(i, j)] * y_large[(i, j)] for (i, j) in hybrid_routes if (i, j) in cost_large_hybrid_route])
)

# Constraints

# 1. Demand Satisfaction for Solo Routes
for i in solo_routes:
    prob += (
        pulp.lpSum([
            (capacity_small_bus - buffer_current_small.get(i, 0)) * x_small[i] if cost_small_bus.get(i) is not None else 0,
            (capacity_large_bus - buffer_current_large.get(i, 0)) * x_large[i] if cost_large_bus.get(i) is not None else 0
        ]) >= demand.get(i, 0)
    )
    print(f"Demand satisfaction constraint for solo route {i}: {(capacity_small_bus - buffer_current_small.get(i, 0)) * x_small[i]} + {(capacity_large_bus - buffer_current_large.get(i, 0)) * x_large[i]} >= {demand.get(i, 0)}")

# 2. Demand Satisfaction for Hybrid Routes
for (i, j) in hybrid_routes:
    prob += (
        pulp.lpSum([
            (capacity_small_bus - buffer_current_small.get((i, j), 0)) * y_small[(i, j)] if cost_small_hybrid_route.get((i, j)) is not None else 0,
            (capacity_large_bus - buffer_current_large.get((i, j), 0)) * y_large[(i, j)] if cost_large_hybrid_route.get((i, j)) is not None else 0
        ]) >= demand.get(i, 0) + demand.get(j, 0)
    )
    print(f"Demand satisfaction constraint for hybrid route ({i}, {j}): {(capacity_small_bus - buffer_current_small.get((i, j), 0)) * y_small[(i, j)]} + {(capacity_large_bus - buffer_current_large.get((i, j), 0)) * y_large[(i, j)]} >= {demand.get(i, 0) + demand.get(j, 0)}")

# 3. Capacity Non-Negativity (implicitly handled by lowBound=0 in variable definitions)

# 4. Zero Demand Constraint
for i in solo_routes:
    if demand[i] == 0:
        prob += x_small[i] == 0
        prob += x_large[i] == 0
        print(f"Zero demand constraint for {i}: x_small[{i}] == 0, x_large[{i}] == 0")

for (i, j) in hybrid_routes:
    if (demand[i] == 0) and (demand[j] == 0):
        prob += y_small[(i, j)] == 0
        prob += y_large[(i, j)] == 0
        print(f"Zero demand constraint for hybrid route ({i}, {j}): y_small[({i}, {j})] == 0, y_large[({i}, {j})] == 0")

        
# 5. Ensure at least one bus is allocated to each route
for i in solo_routes:
    if demand[i] > 0:
        constraint = (x_small[i] + x_large[i] +
                      pulp.lpSum([y_small[(i, j)] for (i, j) in hybrid_routes if i in (i, j)]) +
                      pulp.lpSum([y_large[(i, j)] for (i, j) in hybrid_routes if i in (i, j)])) >= 1
        prob += constraint
        print(f"Minimum allocation constraint for {i}: {constraint}")


# Solve the problem
prob.solve()

# Output the results
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")

print(f"Total Cost = {pulp.value(prob.objective)}")

# Define the problem
prob = pulp.LpProblem("Incoming_Bus_Allocation2", pulp.LpMinimize)

# Decision Variables
x_small = pulp.LpVariable.dicts("x_small", solo_routes, lowBound=0, cat='Integer')
x_large = pulp.LpVariable.dicts("x_large", solo_routes, lowBound=0, cat='Integer')
y_small = pulp.LpVariable.dicts("y_small", hybrid_routes, lowBound=0, cat='Integer')
y_large = pulp.LpVariable.dicts("y_large", hybrid_routes, lowBound=0, cat='Integer')

# Objective Function
prob += (
    pulp.lpSum([cost_small_bus[i] * x_small[i] for i in solo_routes if i in cost_small_bus]) +
    pulp.lpSum([cost_large_bus[i] * x_large[i] for i in solo_routes if i in cost_large_bus]) +
    pulp.lpSum([cost_small_hybrid_route[(i, j)] * y_small[(i, j)] for (i, j) in hybrid_routes if (i, j) in cost_small_hybrid_route]) +
    pulp.lpSum([cost_large_hybrid_route[(i, j)] * y_large[(i, j)] for (i, j) in hybrid_routes if (i, j) in cost_large_hybrid_route])
)

# Constraints

# Ensure that total buses allocated do not exceed the combined demand for shared routes
for i in solo_routes:
    for (j, k) in hybrid_routes:
        if i in (j, k):
            prob += (
                pulp.lpSum([
                    (capacity_small_bus - buffer_current_small.get(i, 0)) * x_small[i] if cost_small_bus.get(i) is not None else 0,
                    (capacity_large_bus - buffer_current_large.get(i, 0)) * x_large[i] if cost_large_bus.get(i) is not None else 0,
                    (capacity_small_bus - buffer_current_small.get((j, k), 0)) * y_small[(j, k)] if cost_small_hybrid_route.get((j, k)) is not None else 0,
                    (capacity_large_bus - buffer_current_large.get((j, k), 0)) * y_large[(j, k)] if cost_large_hybrid_route.get((j, k)) is not None else 0
                ]) >= demand.get(i, 0)
            )
    
# 1. Demand Satisfaction for Solo Routes
for i in solo_routes:
    prob += (
        pulp.lpSum([
            (capacity_small_bus - buffer_current_small.get(i, 0)) * x_small[i] if cost_small_bus.get(i) is not None else 0,
            (capacity_large_bus - buffer_current_large.get(i, 0)) * x_large[i] if cost_large_bus.get(i) is not None else 0
        ]) >= demand.get(i, 0)
    )
    print(f"Demand satisfaction constraint for solo route {i}: {(capacity_small_bus - buffer_current_small.get(i, 0)) * x_small[i]} + {(capacity_large_bus - buffer_current_large.get(i, 0)) * x_large[i]} >= {demand.get(i, 0)}")

# 2. Demand Satisfaction for Hybrid Routes
for (i, j) in hybrid_routes:
    prob += (
        pulp.lpSum([
            (capacity_small_bus - buffer_current_small.get((i, j), 0)) * y_small[(i, j)] if cost_small_hybrid_route.get((i, j)) is not None else 0,
            (capacity_large_bus - buffer_current_large.get((i, j), 0)) * y_large[(i, j)] if cost_large_hybrid_route.get((i, j)) is not None else 0
        ]) >= demand.get(i, 0) + demand.get(j, 0)
    )
    print(f"Demand satisfaction constraint for hybrid route ({i}, {j}): {(capacity_small_bus - buffer_current_small.get((i, j), 0)) * y_small[(i, j)]} + {(capacity_large_bus - buffer_current_large.get((i, j), 0)) * y_large[(i, j)]} >= {demand.get(i, 0) + demand.get(j, 0)}")

    
# 3. Capacity Non-Negativity (implicitly handled by lowBound=0 in variable definitions)

# 4. Zero Demand Constraint
for i in solo_routes:
    if demand[i] == 0:
        prob += x_small[i] == 0
        prob += x_large[i] == 0
        print(f"Zero demand constraint for {i}: x_small[{i}] == 0, x_large[{i}] == 0")

for (i, j) in hybrid_routes:
    if (demand[i] == 0) and (demand[j] == 0):
        prob += y_small[(i, j)] == 0
        prob += y_large[(i, j)] == 0
        print(f"Zero demand constraint for hybrid route ({i}, {j}): y_small[({i}, {j})] == 0, y_large[({i}, {j})] == 0")

        
# 5. Ensure at least one bus is allocated to each route
for i in solo_routes:
    if demand[i] > 0:
        constraint = (x_small[i] + x_large[i] +
                      pulp.lpSum([y_small[(i, j)] for (i, j) in hybrid_routes if i in (i, j)]) +
                      pulp.lpSum([y_large[(i, j)] for (i, j) in hybrid_routes if i in (i, j)])) >= 1
        prob += constraint
        print(f"Minimum allocation constraint for {i}: {constraint}")


# Solve the problem
prob.solve()

# Output the results
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")

print(f"Total Cost = {pulp.value(prob.objective)}")

# Define decision variables (assuming these are already defined in your original code)
x_small = pulp.LpVariable.dicts("x_small", solo_routes, lowBound=0, cat='Integer')
x_large = pulp.LpVariable.dicts("x_large", solo_routes, lowBound=0, cat='Integer')
y_small = pulp.LpVariable.dicts("y_small", hybrid_routes, lowBound=0, cat='Integer')
y_large = pulp.LpVariable.dicts("y_large", hybrid_routes, lowBound=0, cat='Integer')

# Define adjusted demand variables
adjusted_demand = {route: pulp.LpVariable(f"adjusted_demand_{route}", lowBound=0, cat='Continuous') for route in solo_routes}

# Solo Route Demand Satisfaction
for i in solo_routes:
    prob += (
        pulp.lpSum([
            (capacity_small_bus - buffer_current_small.get(i, 0)) * x_small[i] if cost_small_bus.get(i) is not None else 0,
            (capacity_large_bus - buffer_current_large.get(i, 0)) * x_large[i] if cost_large_bus.get(i) is not None else 0
        ]) >= demand.get(i, 0)
    )

# Hybrid Route Demand Satisfaction and Adjustments
for (i, j) in hybrid_routes:
    hybrid_demand = demand.get(i, 0) + demand.get(j, 0)
    
    prob += (
        pulp.lpSum([
            (capacity_small_bus - buffer_current_small.get((i, j), 0)) * y_small[(i, j)] if cost_small_hybrid_route.get((i, j)) is not None else 0,
            (capacity_large_bus - buffer_current_large.get((i, j), 0)) * y_large[(i, j)] if cost_large_hybrid_route.get((i, j)) is not None else 0
        ]) >= hybrid_demand
    )
    
    # Adjust solo route demands
    prob += adjusted_demand[i] == demand.get(i, 0) - (capacity_small_bus - buffer_current_small.get((i, j), 0)) * y_small[(i, j)] / 2
    prob += adjusted_demand[j] == demand.get(j, 0) - (capacity_small_bus - buffer_current_small.get((i, j), 0)) * y_small[(i, j)] / 2

# Ensure adjusted demands are non-negative
for i in adjusted_demand:
    prob += adjusted_demand[i] >= 0

# Solo Route Demand Satisfaction with Adjusted Demands
for i in solo_routes:
    prob += (
        pulp.lpSum([
            (capacity_small_bus - buffer_current_small.get(i, 0)) * x_small[i] if cost_small_bus.get(i) is not None else 0,
            (capacity_large_bus - buffer_current_large.get(i, 0)) * x_large[i] if cost_large_bus.get(i) is not None else 0
        ]) >= adjusted_demand[i]
    )

# Define the objective function (example, replace with your actual objective function)
prob += pulp.lpSum([cost_small_bus.get(i, 0) * x_small[i] + cost_large_bus.get(i, 0) * x_large[i] for i in solo_routes]) + \
        pulp.lpSum([cost_small_hybrid_route.get((i, j), 0) * y_small[(i, j)] + cost_large_hybrid_route.get((i, j), 0) * y_large[(i, j)] for (i, j) in hybrid_routes])

# Solve the problem
prob.solve()

# Print the results
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")

print(f"Total Cost = {pulp.value(prob.objective)}")

# Define decision variables (assuming these are already defined in your original code)
x_small = pulp.LpVariable.dicts("x_small", solo_routes, lowBound=0, cat='Integer')
x_large = pulp.LpVariable.dicts("x_large", solo_routes, lowBound=0, cat='Integer')
y_small = pulp.LpVariable.dicts("y_small", hybrid_routes, lowBound=0, cat='Integer')
y_large = pulp.LpVariable.dicts("y_large", hybrid_routes, lowBound=0, cat='Integer')

# Adjusted demands for solo routes
adjusted_demand = {route: pulp.LpVariable(f"adjusted_demand_{route}", lowBound=0, cat='Continuous') for route in solo_routes}

# Solo Route Demand Satisfaction
for i in solo_routes:
    prob += (
        pulp.lpSum([
            (capacity_small_bus - buffer_current_small.get(i, 0)) * x_small[i] if cost_small_bus.get(i) is not None else 0,
            (capacity_large_bus - buffer_current_large.get(i, 0)) * x_large[i] if cost_large_bus.get(i) is not None else 0
        ]) >= demand.get(i, 0)
    )

# Hybrid Route Demand Satisfaction and Adjustments
for (i, j) in hybrid_routes:
    hybrid_demand = demand.get(i, 0) + demand.get(j, 0)
    
    prob += (
        pulp.lpSum([
            (capacity_small_bus - buffer_current_small.get((i, j), 0)) * y_small[(i, j)] if cost_small_hybrid_route.get((i, j)) is not None else 0,
            (capacity_large_bus - buffer_current_large.get((i, j), 0)) * y_large[(i, j)] if cost_large_hybrid_route.get((i, j)) is not None else 0
        ]) >= hybrid_demand
    )
    
    # Adjust solo route demands based on hybrid bus allocations
    adjusted_demand[i] -= pulp.lpSum([
        (capacity_small_bus - buffer_current_small.get((i, j), 0)) * y_small[(i, j)] / 2 if cost_small_hybrid_route.get((i, j)) is not None else 0,
        (capacity_large_bus - buffer_current_large.get((i, j), 0)) * y_large[(i, j)] / 2 if cost_large_hybrid_route.get((i, j)) is not None else 0
    ])
    
    adjusted_demand[j] -= pulp.lpSum([
        (capacity_small_bus - buffer_current_small.get((i, j), 0)) * y_small[(i, j)] / 2 if cost_small_hybrid_route.get((i, j)) is not None else 0,
        (capacity_large_bus - buffer_current_large.get((i, j), 0)) * y_large[(i, j)] / 2 if cost_large_hybrid_route.get((i, j)) is not None else 0
    ])

# Ensure adjusted demands are non-negative
for i in adjusted_demand:
    prob += adjusted_demand[i] >= 0

# Solo Route Demand Satisfaction with Adjusted Demands
for i in solo_routes:
    prob += (
        pulp.lpSum([
            (capacity_small_bus - buffer_current_small.get(i, 0)) * x_small[i] if cost_small_bus.get(i) is not None else 0,
            (capacity_large_bus - buffer_current_large.get(i, 0)) * x_large[i] if cost_large_bus.get(i) is not None else 0
        ]) >= adjusted_demand[i]
    )

# Define the objective function (example, replace with your actual objective function)
prob += pulp.lpSum([cost_small_bus.get(i, 0) * x_small[i] + cost_large_bus.get(i, 0) * x_large[i] for i in solo_routes]) + \
        pulp.lpSum([cost_small_hybrid_route.get((i, j), 0) * y_small[(i, j)] + cost_large_hybrid_route.get((i, j), 0) * y_large[(i, j)] for (i, j) in hybrid_routes])

# Solve the problem
prob.solve()

# Print the results
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")

print(f"Total Cost = {pulp.value(prob.objective)}")

# Define decision variables (assuming these are already defined in your original code)
x_small = pulp.LpVariable.dicts("x_small", solo_routes, lowBound=0, cat='Integer')
x_large = pulp.LpVariable.dicts("x_large", solo_routes, lowBound=0, cat='Integer')
y_small = pulp.LpVariable.dicts("y_small", hybrid_routes, lowBound=0, cat='Integer')
y_large = pulp.LpVariable.dicts("y_large", hybrid_routes, lowBound=0, cat='Integer')

# Solo Route Demand Satisfaction
for i in solo_routes:
    prob += (
        pulp.lpSum([
            (capacity_small_bus - buffer_current_small.get(i, 0)) * x_small[i] if cost_small_bus.get(i) is not None else 0,
            (capacity_large_bus - buffer_current_large.get(i, 0)) * x_large[i] if cost_large_bus.get(i) is not None else 0
        ]) >= demand.get(i, 0)
    )

# Hybrid Route Demand Satisfaction
for (i, j) in hybrid_routes:
    prob += (
        pulp.lpSum([
            (capacity_small_bus - buffer_current_small.get((i, j), 0)) * y_small[(i, j)] if cost_small_hybrid_route.get((i, j)) is not None else 0,
            (capacity_large_bus - buffer_current_large.get((i, j), 0)) * y_large[(i, j)] if cost_large_hybrid_route.get((i, j)) is not None else 0
        ]) >= demand.get(i, 0) + demand.get(j, 0)
    )

# Ensure adjusted demands are non-negative
for i in solo_routes:
    prob += (
        pulp.lpSum([
            (capacity_small_bus - buffer_current_small.get(i, 0)) * x_small[i] if cost_small_bus.get(i) is not None else 0,
            (capacity_large_bus - buffer_current_large.get(i, 0)) * x_large[i] if cost_large_bus.get(i) is not None else 0
        ]) + pulp.lpSum([
            (capacity_small_bus - buffer_current_small.get((i, j), 0)) * y_small[(i, j)] if (i, j) in hybrid_routes and cost_small_hybrid_route.get((i, j)) is not None else 0,
            (capacity_large_bus - buffer_current_large.get((i, j), 0)) * y_large[(i, j)] if (i, j) in hybrid_routes and cost_large_hybrid_route.get((i, j)) is not None else 0
        ]) <= demand.get(i, 0) + 10  # Allow a small buffer to prevent over-restricting
    )

# Define the objective function (example, replace with your actual objective function)
prob += pulp.lpSum([cost_small_bus.get(i, 0) * x_small[i] + cost_large_bus.get(i, 0) * x_large[i] for i in solo_routes]) + \
        pulp.lpSum([cost_small_hybrid_route.get((i, j), 0) * y_small[(i, j)] + cost_large_hybrid_route.get((i, j), 0) * y_large[(i, j)] for (i, j) in hybrid_routes])

# Solve the problem
prob.solve()

# Print the results
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")

print(f"Total Cost = {pulp.value(prob.objective)}")

# Demand Constraints for Solo Routes
for i in solo_routes:
    prob += (
        (capacity_small_bus - buffer_current_small[i]) * x_small[i] +
        (capacity_large_bus - buffer_current_large[i]) * x_large[i] +
        pulp.lpSum([
            (capacity_small_bus - buffer_current_small[(i, j)]) * y_small[(i, j)] +
            (capacity_large_bus - buffer_current_large[(i, j)]) * y_large[(i, j)]
            for (i_, j) in hybrid_routes if i_ == i
        ])
        >= demand[i], f"Demand_Solo_{i}"
    )

# Demand Constraints for Hybrid Routes
for (i, j) in hybrid_routes:
    prob += (
        (capacity_small_bus - buffer_current_small[(i, j)]) * y_small[(i, j)] +
        (capacity_large_bus - buffer_current_large[(i, j)]) * y_large[(i, j)] +
        (capacity_small_bus - buffer_current_small[i]) * x_small[i] +
        (capacity_large_bus - buffer_current_large[i]) * x_large[i] +
        (capacity_small_bus - buffer_current_small[j]) * x_small[j] +
        (capacity_large_bus - buffer_current_large[j]) * x_large[j]
        >= demand[i] + demand[j], f"Demand_Hybrid_{i}_{j}"
    )

# Solve the problem
prob.solve()

# Print the results
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")

# Objective Value
print(f"Total Cost = {pulp.value(prob.objective)}")


# Demand Constraints for Solo Routes
for i in solo_routes:
    if cost_small_bus.get(i) is not None or cost_large_bus.get(i) is not None:
        prob += (
            (capacity_small_bus - buffer_current_small[i]) * x_small[i] +
            (capacity_large_bus - buffer_current_large[i]) * x_large[i] +
            pulp.lpSum([
                (capacity_small_bus - buffer_current_small[(i, j)]) * y_small[(i, j)] +
                (capacity_large_bus - buffer_current_large[(i, j)]) * y_large[(i, j)]
                for (i_, j) in hybrid_routes if i_ == i and (cost_small_hybrid_route.get((i, j)) is not None or cost_large_hybrid_route.get((i, j)) is not None)
            ])
            >= demand.get(i, 0), f"Demand_Solo_{i}"
        )

# Demand Constraints for Hybrid Routes
for (i, j) in hybrid_routes:
    if cost_small_hybrid_route.get((i, j)) is not None or cost_large_hybrid_route.get((i, j)) is not None:
        prob += (
            (capacity_small_bus - buffer_current_small[(i, j)]) * y_small[(i, j)] +
            (capacity_large_bus - buffer_current_large[(i, j)]) * y_large[(i, j)] +
            (capacity_small_bus - buffer_current_small[i]) * x_small[i] * (1 if cost_small_bus.get(i) is not None else 0) +
            (capacity_large_bus - buffer_current_large[i]) * x_large[i] * (1 if cost_large_bus.get(i) is not None else 0) +
            (capacity_small_bus - buffer_current_small[j]) * x_small[j] * (1 if cost_small_bus.get(j) is not None else 0) +
            (capacity_large_bus - buffer_current_large[j]) * x_large[j] * (1 if cost_large_bus.get(j) is not None else 0)
            >= demand.get(i, 0) + demand.get(j, 0), f"Demand_Hybrid_{i}_{j}"
        )

# Solve the problem
prob.solve()

# Print the results
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")

# Objective Value
print(f"Total Cost = {pulp.value(prob.objective)}")


# Demand Constraints for Solo Routes
'''for i in solo_routes:
    prob += (
        (capacity_small_bus - buffer_current_small[i]) * x_small[i] if cost_small_bus.get(i) is not None else 0 +
        (capacity_large_bus - buffer_current_large[i]) * x_large[i] if cost_large_bus.get(i) is not None else 0 +
        pulp.lpSum([(capacity_small_bus - buffer_current_small[(i, j)]) * y_small[(i, j)] for (i_, j) in hybrid_routes if i_ == i and (cost_small_hybrid_route.get(i_, j) is not None else 0)]) +
        pulp.lpSum([(capacity_large_bus - buffer_current_large[(i, j)]) * y_large[(i, j)] for (i_, j) in hybrid_routes if i_ == i and (cost_large_hybrid_route.get(i_, j) is not None else 0)])
        >= demand.get(i, 0), f"Demand_Solo_{i}"
    )'''
    
# Demand Constraints for Solo Routes
for i in solo_routes:
    prob += (
        (capacity_small_bus - buffer_current_small[i]) * x_small[i] if cost_small_bus.get(i) is not None else 0 +
        (capacity_large_bus - buffer_current_large[i]) * x_large[i] if cost_large_bus.get(i) is not None else 0 +
        pulp.lpSum([(capacity_small_bus - buffer_current_small[(i, j)]) * y_small[(i, j)] for (i_, j) in hybrid_routes if i_ == i and cost_small_hybrid_route.get((i_, j)) is not None]) +
        pulp.lpSum([(capacity_large_bus - buffer_current_large[(i, j)]) * y_large[(i, j)] for (i_, j) in hybrid_routes if i_ == i and cost_large_hybrid_route.get((i_, j)) is not None])
        >= demand.get(i, 0), f"Demand_Solo_{i}"
    )


# Demand Constraints for Hybrid Routes
for (i, j) in hybrid_routes:
    if cost_small_hybrid_route.get((i, j)) is not None or cost_large_hybrid_route.get((i, j)) is not None:
        prob += (
            (capacity_small_bus - buffer_current_small[(i, j)]) * y_small[(i, j)] +
            (capacity_large_bus - buffer_current_large[(i, j)]) * y_large[(i, j)] +
            (capacity_small_bus - buffer_current_small[i]) * x_small[i] +
            (capacity_large_bus - buffer_current_large[i]) * x_large[i] +
            (capacity_small_bus - buffer_current_small[j]) * x_small[j] +
            (capacity_large_bus - buffer_current_large[j]) * x_large[j]
            >= demand.get(i, 0) + demand.get(j, 0), f"Demand_Hybrid_{i}_{j}"
        )

# Solve the problem
prob.solve()

# Print the results
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")

# Objective Value
print(f"Total Cost = {pulp.value(prob.objective)}")


# Demand Constraints for Solo Routes
for i in solo_routes:
    if cost_small_bus.get(i) is not None:
        prob += (
            (capacity_small_bus - buffer_current_small[i]) * x_small[i] +
            pulp.lpSum([
                (capacity_small_bus - buffer_current_small[(i, j)]) * y_small[(i, j)]
                for (i_, j) in hybrid_routes if i_ == i and cost_small_hybrid_route.get((i, j)) is not None
            ])
            >= demand.get(i, 0), f"Demand_Solo_Small_{i}"
        )
    if cost_large_bus.get(i) is not None:
        prob += (
            (capacity_large_bus - buffer_current_large[i]) * x_large[i] +
            pulp.lpSum([
                (capacity_large_bus - buffer_current_large[(i, j)]) * y_large[(i, j)]
                for (i_, j) in hybrid_routes if i_ == i and cost_large_hybrid_route.get((i, j)) is not None
            ])
            >= demand.get(i, 0), f"Demand_Solo_Large_{i}"
        )

# Demand Constraints for Hybrid Routes
for (i, j) in hybrid_routes:
    if cost_small_hybrid_route.get((i, j)) is not None:
        prob += (
            (capacity_small_bus - buffer_current_small[(i, j)]) * y_small[(i, j)] +
            (capacity_small_bus - buffer_current_small[i]) * x_small[i] * (1 if cost_small_bus.get(i) is not None else 0) +
            (capacity_small_bus - buffer_current_small[j]) * x_small[j] * (1 if cost_small_bus.get(j) is not None else 0)
            >= demand.get(i, 0) + demand.get(j, 0), f"Demand_Hybrid_Small_{i}_{j}"
        )
    if cost_large_hybrid_route.get((i, j)) is not None:
        prob += (
            (capacity_large_bus - buffer_current_large[(i, j)]) * y_large[(i, j)] +
            (capacity_large_bus - buffer_current_large[i]) * x_large[i] * (1 if cost_large_bus.get(i) is not None else 0) +
            (capacity_large_bus - buffer_current_large[j]) * x_large[j] * (1 if cost_large_bus.get(j) is not None else 0)
            >= demand.get(i, 0) + demand.get(j, 0), f"Demand_Hybrid_Large_{i}_{j}"
        )

# Solve the problem
prob.solve()

# Print the results
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")

# Objective Value
print(f"Total Cost = {pulp.value(prob.objective)}")

# Demand Constraints for Hybrid Routes
for (i, j) in hybrid_routes:
    if cost_small_hybrid_route.get((i, j)) is not None:
        prob += (
            (capacity_small_bus - buffer_current_small[(i, j)]) * y_small[(i, j)] +
            (capacity_small_bus - buffer_current_small[i]) * x_small[i] * (1 if cost_small_bus.get(i) is not None else 0) +
            (capacity_small_bus - buffer_current_small[j]) * x_small[j] * (1 if cost_small_bus.get(j) is not None else 0)
            >= demand.get(i, 0) + demand.get(j, 0), f"Demand_Hybrid_Small_{i}_{j}"
        )
    if cost_large_hybrid_route.get((i, j)) is not None:
        prob += (
            (capacity_large_bus - buffer_current_large[(i, j)]) * y_large[(i, j)] +
            (capacity_large_bus - buffer_current_large[i]) * x_large[i] * (1 if cost_large_bus.get(i) is not None else 0) +
            (capacity_large_bus - buffer_current_large[j]) * x_large[j] * (1 if cost_large_bus.get(j) is not None else 0)
            >= demand.get(i, 0) + demand.get(j, 0), f"Demand_Hybrid_Large_{i}_{j}"
        )

# Solve the problem
prob.solve()

# Print the results
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")

# Objective Value
print(f"Total Cost = {pulp.value(prob.objective)}")

# Objective Function
prob += (
    pulp.lpSum([cost_small_bus[i] * x_small[i] for i in solo_routes if i in cost_small_bus]) +
    pulp.lpSum([cost_large_bus[i] * x_large[i] for i in solo_routes if i in cost_large_bus]) +
    pulp.lpSum([cost_small_hybrid_route[(i, j)] * y_small[(i, j)] for (i, j) in hybrid_routes if (i, j) in cost_small_hybrid_route]) +
    pulp.lpSum([cost_large_hybrid_route[(i, j)] * y_large[(i, j)] for (i, j) in hybrid_routes if (i, j) in cost_large_hybrid_route])
)

# Demand Constraints for Solo Routes
for i in solo_routes:
    if cost_small_bus.get(i) is not None:
        prob += (
            (capacity_small_bus - buffer_current_small[i]) * x_small[i] +
            pulp.lpSum([
                (capacity_small_bus - buffer_current_small[(i, j)]) * y_small[(i, j)]
                for (i_, j) in hybrid_routes if i_ == i and cost_small_hybrid_route.get((i, j)) is not None
            ])
            >= demand.get(i, 0), f"Demand_Solo_Small_{i}"
        )
    if cost_large_bus.get(i) is not None:
        prob += (
            (capacity_large_bus - buffer_current_large[i]) * x_large[i] +
            pulp.lpSum([
                (capacity_large_bus - buffer_current_large[(i, j)]) * y_large[(i, j)]
                for (i_, j) in hybrid_routes if i_ == i and cost_large_hybrid_route.get((i, j)) is not None
            ])
            >= demand.get(i, 0), f"Demand_Solo_Large_{i}"
        )

# Demand Constraints for Hybrid Routes
for (i, j) in hybrid_routes:
    if cost_small_hybrid_route.get((i, j)) is not None:
        prob += (
            (capacity_small_bus - buffer_current_small[(i, j)]) * y_small[(i, j)] +
            (capacity_small_bus - buffer_current_small[i]) * x_small[i] * (1 if cost_small_bus.get(i) is not None else 0) +
            (capacity_small_bus - buffer_current_small[j]) * x_small[j] * (1 if cost_small_bus.get(j) is not None else 0)
            >= demand.get(i, 0) + demand.get(j, 0), f"Demand_Hybrid_Small_{i}_{j}"
        )
    if cost_large_hybrid_route.get((i, j)) is not None:
        prob += (
            (capacity_large_bus - buffer_current_large[(i, j)]) * y_large[(i, j)] +
            (capacity_large_bus - buffer_current_large[i]) * x_large[i] * (1 if cost_large_bus.get(i) is not None else 0) +
            (capacity_large_bus - buffer_current_large[j]) * x_large[j] * (1 if cost_large_bus.get(j) is not None else 0)
            >= demand.get(i, 0) + demand.get(j, 0), f"Demand_Hybrid_Large_{i}_{j}"
        )

# Solve the problem
prob.solve()

# Print the results
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")

# Objective Value
print(f"Total Cost = {pulp.value(prob.objective)}")

# Demand Constraints for Solo Routes
for i in solo_routes:
    if cost_small_bus.get(i) is not None:
        prob += (
            (capacity_small_bus - buffer_current_small[i]) * x_small[i] >= demand.get(i, 0), 
            f"Demand_Solo_Small_{i}"
        )
    if cost_large_bus.get(i) is not None:
        prob += (
            (capacity_large_bus - buffer_current_large[i]) * x_large[i] >= demand.get(i, 0), 
            f"Demand_Solo_Large_{i}"
        )

# Demand Constraints for Hybrid Routes
for (i, j) in hybrid_routes:
    if cost_small_hybrid_route.get((i, j)) is not None:
        prob += (
            (capacity_small_bus - buffer_current_small[(i, j)]) * y_small[(i, j)] +
            (capacity_small_bus - buffer_current_small[i]) * x_small[i] * (1 if cost_small_bus.get(i) is not None else 0) +
            (capacity_small_bus - buffer_current_small[j]) * x_small[j] * (1 if cost_small_bus.get(j) is not None else 0)
            >= demand.get(i, 0) + demand.get(j, 0), 
            f"Demand_Hybrid_Small_{i}_{j}"
        )
    if cost_large_hybrid_route.get((i, j)) is not None:
        prob += (
            (capacity_large_bus - buffer_current_large[(i, j)]) * y_large[(i, j)] +
            (capacity_large_bus - buffer_current_large[i]) * x_large[i] * (1 if cost_large_bus.get(i) is not None else 0) +
            (capacity_large_bus - buffer_current_large[j]) * x_large[j] * (1 if cost_large_bus.get(j) is not None else 0)
            >= demand.get(i, 0) + demand.get(j, 0), 
            f"Demand_Hybrid_Large_{i}_{j}"
        )

# Solve the problem
prob.solve()

# Print the results
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")

# Objective Value
print(f"Total Cost = {pulp.value(prob.objective)}")

# Define the variables
x_small_solo = pulp.LpVariable.dicts("x_small_solo", solo_routes, lowBound=0, cat='Integer')
x_large_solo = pulp.LpVariable.dicts("x_large_solo", solo_routes, lowBound=0, cat='Integer')
x_small_hybrid = pulp.LpVariable.dicts("x_small_hybrid", hybrid_routes, lowBound=0, cat='Integer')
x_large_hybrid = pulp.LpVariable.dicts("x_large_hybrid", hybrid_routes, lowBound=0, cat='Integer')

# Define the objective function
prob += pulp.lpSum([cost_small_bus[i] * x_small_solo[i] for i in solo_routes if i in cost_small_bus]) + \
        pulp.lpSum([cost_large_bus[i] * x_large_solo[i] for i in solo_routes if i in cost_large_bus]) + \
        pulp.lpSum([cost_small_hybrid_route[i] * x_small_hybrid[i] for i in hybrid_routes if i in cost_small_hybrid_route]) + \
        pulp.lpSum([cost_large_hybrid_route[i] * x_large_hybrid[i] for i in hybrid_routes if i in cost_large_hybrid_route])

# Define the constraints
for i in solo_routes:
    prob += (capacity_small_bus - buffer_current_small[i]) * x_small_solo[i] + \
            (capacity_large_bus - buffer_current_large[i]) * x_large_solo[i] + \
            pulp.lpSum([(capacity_small_bus - buffer_current_small[(i,j)]) * x_small_hybrid[(i,j)] for j in solo_routes if (i,j) in hybrid_routes and (i,j) in cost_small_hybrid_route]) + \
            pulp.lpSum([(capacity_large_bus - buffer_current_large[(i,j)]) * x_large_hybrid[(i,j)] for j in solo_routes if (i,j) in hybrid_routes and (i,j) in cost_large_hybrid_route]) >= demand[i]

for (i,j) in hybrid_routes:
    if (i,j) in cost_small_hybrid_route or (i,j) in cost_large_hybrid_route:
        prob += (capacity_small_bus - buffer_current_small[(i,j)]) * x_small_hybrid[(i,j)] + \
                (capacity_large_bus - buffer_current_large[(i,j)]) * x_large_hybrid[(i,j)] >= demand.get((i,j))

# Solve the problem
prob.solve()

# Output results
for v in prob.variables():
    print(v.name, "=", v.varValue)

# Define the variables
x_small_solo = pulp.LpVariable.dicts("x_small_solo", solo_routes, lowBound=0, cat='Integer')
x_large_solo = pulp.LpVariable.dicts("x_large_solo", solo_routes, lowBound=0, cat='Integer')
x_small_hybrid = pulp.LpVariable.dicts("x_small_hybrid", hybrid_routes, lowBound=0, cat='Integer')
x_large_hybrid = pulp.LpVariable.dicts("x_large_hybrid", hybrid_routes, lowBound=0, cat='Integer')

# Define the objective function
prob += pulp.lpSum([cost_small_bus[i] * x_small_solo[i] for i in solo_routes if i in cost_small_bus]) + \
        pulp.lpSum([cost_large_bus[i] * x_large_solo[i] for i in solo_routes if i in cost_large_bus]) + \
        pulp.lpSum([cost_small_hybrid_route[i] * x_small_hybrid[i] for i in hybrid_routes if i in cost_small_hybrid_route]) + \
        pulp.lpSum([cost_large_hybrid_route[i] * x_large_hybrid[i] for i in hybrid_routes if i in cost_large_hybrid_route])

# Define the constraints
for i in solo_routes:
    if i in cost_small_bus or i in cost_large_bus:
        prob += (capacity_small_bus - buffer_current_small[i]) * x_small_solo[i] + \
                (capacity_large_bus - buffer_current_large[i]) * x_large_solo[i] + \
                pulp.lpSum([(capacity_small_bus - buffer_current_small[(i,j)]) * x_small_hybrid[(i,j)] for j in solo_routes if (i,j) in hybrid_routes and (i,j) in cost_small_hybrid_route]) + \
                pulp.lpSum([(capacity_large_bus - buffer_current_large[(i,j)]) * x_large_hybrid[(i,j)] for j in solo_routes if (i,j) in hybrid_routes and (i,j) in cost_large_hybrid_route]) >= demand[i]

for (i,j) in hybrid_routes:
    if (i,j) in cost_small_hybrid_route or (i,j) in cost_large_hybrid_route:
        prob += (capacity_small_bus - buffer_current_small[(i,j)]) * x_small_hybrid[(i,j)] + \
                (capacity_large_bus - buffer_current_large[(i,j)]) * x_large_hybrid[(i,j)] >= demand[(i,j)]

# Solve the problem
prob.solve()

# Output results
for v in prob.variables():
    print(v.name, "=", v.varValue)


# Define the variables
x_small_solo = pulp.LpVariable.dicts("x_small_solo", solo_routes, lowBound=0, cat='Integer')
x_large_solo = pulp.LpVariable.dicts("x_large_solo", solo_routes, lowBound=0, cat='Integer')
x_small_hybrid = pulp.LpVariable.dicts("x_small_hybrid", hybrid_routes, lowBound=0, cat='Integer')
x_large_hybrid = pulp.LpVariable.dicts("x_large_hybrid", hybrid_routes, lowBound=0, cat='Integer')

# Define the objective function
prob += pulp.lpSum([cost_small_bus[i] * x_small_solo[i] for i in solo_routes if i in cost_small_bus]) + \
        pulp.lpSum([cost_large_bus[i] * x_large_solo[i] for i in solo_routes if i in cost_large_bus]) + \
        pulp.lpSum([cost_small_hybrid_route[i] * x_small_hybrid[i] for i in hybrid_routes if i in cost_small_hybrid_route]) + \
        pulp.lpSum([cost_large_hybrid_route[i] * x_large_hybrid[i] for i in hybrid_routes if i in cost_large_hybrid_route])

# Define the constraints
for i in solo_routes:
    if i in cost_small_bus:
        prob += (capacity_small_bus - buffer_current_small[i]) * x_small_solo[i] + \
                pulp.lpSum([(capacity_small_bus - buffer_current_small[(i,j)]) * x_small_hybrid[(i,j)] for j in solo_routes if (i,j) in hybrid_routes and (i,j) in cost_small_hybrid_route]) >= demand[i]
    if i in cost_large_bus:
        prob += (capacity_large_bus - buffer_current_large[i]) * x_large_solo[i] + \
                pulp.lpSum([(capacity_large_bus - buffer_current_large[(i,j)]) * x_large_hybrid[(i,j)] for j in solo_routes if (i,j) in hybrid_routes and (i,j) in cost_large_hybrid_route]) >= demand[i]

for (i,j) in hybrid_routes:
    if (i,j) in cost_small_hybrid_route:
        prob += (capacity_small_bus - buffer_current_small[(i,j)]) * x_small_hybrid[(i,j)] >= demand[(i,j)]
    if (i,j) in cost_large_hybrid_route:
        prob += (capacity_large_bus - buffer_current_large[(i,j)]) * x_large_hybrid[(i,j)] >= demand[(i,j)]

# Solve the problem
prob.solve()

# Output results
for v in prob.variables():
    print(v.name, "=", v.varValue)

# Define the variables
x_small_solo = pulp.LpVariable.dicts("x_small_solo", solo_routes, lowBound=0, cat='Integer')
x_large_solo = pulp.LpVariable.dicts("x_large_solo", solo_routes, lowBound=0, cat='Integer')
x_small_hybrid = pulp.LpVariable.dicts("x_small_hybrid", hybrid_routes, lowBound=0, cat='Integer')
x_large_hybrid = pulp.LpVariable.dicts("x_large_hybrid", hybrid_routes, lowBound=0, cat='Integer')

# Define the objective function
prob += pulp.lpSum([cost_small_bus[i] * x_small_solo[i] for i in solo_routes if i in cost_small_bus]) + \
        pulp.lpSum([cost_large_bus[i] * x_large_solo[i] for i in solo_routes if i in cost_large_bus]) + \
        pulp.lpSum([cost_small_hybrid_route[i] * x_small_hybrid[i] for i in hybrid_routes if i in cost_small_hybrid_route]) + \
        pulp.lpSum([cost_large_hybrid_route[i] * x_large_hybrid[i] for i in hybrid_routes if i in cost_large_hybrid_route])

# Define the constraints
for i in solo_routes:
    if i in cost_small_bus:
        prob += (capacity_small_bus - buffer_current_small[i]) * x_small_solo[i] + \
                pulp.lpSum([(capacity_small_bus - buffer_current_small[(i,j)]) * x_small_hybrid[(i,j)] for j in solo_routes if (i,j) in hybrid_routes and (i,j) in cost_small_hybrid_route]) >= demand[i]
    if i in cost_large_bus:
        prob += (capacity_large_bus - buffer_current_large[i]) * x_large_solo[i] + \
                pulp.lpSum([(capacity_large_bus - buffer_current_large[(i,j)]) * x_large_hybrid[(i,j)] for j in solo_routes if (i,j) in hybrid_routes and (i,j) in cost_large_hybrid_route]) >= demand[i]

for (i,j) in hybrid_routes:
    combined_demand = demand[i] + demand[j]
    if (i,j) in cost_small_hybrid_route:
        prob += (capacity_small_bus - buffer_current_small[(i,j)]) * x_small_hybrid[(i,j)] >= combined_demand - \
                ((capacity_small_bus - buffer_current_small[i]) * x_small_solo[i] + (capacity_small_bus - buffer_current_small[j]) * x_small_solo[j])
    if (i,j) in cost_large_hybrid_route:
        prob += (capacity_large_bus - buffer_current_large[(i,j)]) * x_large_hybrid[(i,j)] >= combined_demand - \
                ((capacity_large_bus - buffer_current_large[i]) * x_large_solo[i] + (capacity_large_bus - buffer_current_large[j]) * x_large_solo[j])

# Solve the problem
prob.solve()

# Output results
for v in prob.variables():
    print(v.name, "=", v.varValue)

In [9]:
# Decision Variables
x_small = pulp.LpVariable.dicts("x_small", solo_routes, lowBound=0, cat='Integer')
x_large = pulp.LpVariable.dicts("x_large", solo_routes, lowBound=0, cat='Integer')
y_small = pulp.LpVariable.dicts("y_small", hybrid_routes, lowBound=0, cat='Integer')
y_large = pulp.LpVariable.dicts("y_large", hybrid_routes, lowBound=0, cat='Integer')

# Objective Function
prob += (
    pulp.lpSum([cost_small_bus[i] * x_small[i] for i in solo_routes if cost_small_bus.get(i, 0) > 0]) +
    pulp.lpSum([cost_large_bus[i] * x_large[i] for i in solo_routes if cost_large_bus.get(i, 0) > 0]) +
    pulp.lpSum([cost_small_hybrid_route.get((i, j), 0) * y_small[(i, j)] for (i, j) in hybrid_routes if cost_small_hybrid_route.get((i, j), 0) > 0]) +
    pulp.lpSum([cost_large_hybrid_route.get((i, j), 0) * y_large[(i, j)] for (i, j) in hybrid_routes if cost_large_hybrid_route.get((i, j), 0) > 0])
)

# Constraints
for i in solo_routes:
    prob += (
        (capacity_small_bus * x_small[i] if cost_small_bus.get(i, 0) > 0 else 0) +
        (capacity_large_bus * x_large[i] if cost_large_bus.get(i, 0) > 0 else 0) +
        pulp.lpSum([capacity_small_bus * y_small[(i, j)] for j in solo_routes if cost_small_hybrid_route.get((i, j)) is not None]) +
        pulp.lpSum([capacity_large_bus * y_large[(i, j)] for j in solo_routes if cost_large_hybrid_route.get((i, j)) is not None]) +
        pulp.lpSum([capacity_small_bus * y_small[(j, i)] for j in solo_routes if cost_small_hybrid_route.get((j, i)) is not None]) +
        pulp.lpSum([capacity_large_bus * y_large[(j, i)] for j in solo_routes if cost_large_hybrid_route.get((j, i)) is not None])
        >= demand.get(i, 0), f"Demand_Constraint_{i}"
    )

# Solve the problem
prob.solve()

# Print the results
for v in prob.variables():
    print(v.name, "=", v.varValue)

print("Total Cost = ", pulp.value(prob.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/hk/l0w5001d7pdc860p4ssywh3h0000gn/T/eee1eab5dc45477794f8c7b7dc1c60a1-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/hk/l0w5001d7pdc860p4ssywh3h0000gn/T/eee1eab5dc45477794f8c7b7dc1c60a1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 86 RHS
At line 93 BOUNDS
At line 111 ENDATA
Problem MODEL has 6 rows, 17 columns and 23 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1710.86 - 0.00 seconds
Cgl0003I 0 fixed, 17 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0003I 0 

In [10]:
for name, constraint in prob.constraints.items():
    print(f"Constraint {name}: {constraint}")

Constraint Demand_Constraint_ALABANG: 56*x_large_ALABANG + 18*x_small_ALABANG + 56*y_large_('ALABANG',_'CARMONA') + 18*y_small_('ALABANG',_'CARMONA') >= 4
Constraint Demand_Constraint_BINAN: 56*x_large_BINAN + 18*x_small_BINAN + 56*y_large_('BINAN',_'CARMONA') + 18*y_small_('BINAN',_'CARMONA') >= 6
Constraint Demand_Constraint_CARMONA: 56*x_large_CARMONA + 56*y_large_('ALABANG',_'CARMONA') + 56*y_large_('BINAN',_'CARMONA') + 18*y_small_('ALABANG',_'CARMONA') + 18*y_small_('BINAN',_'CARMONA') >= 4
Constraint Demand_Constraint_BALIBAGO: 56*x_large_BALIBAGO + 18*x_small_BALIBAGO >= 31
Constraint Demand_Constraint_CABUYAO: 56*x_large_CABUYAO + 18*x_small_CABUYAO + 56*y_large_('CALAMBA',_'CABUYAO') + 18*y_small_('CALAMBA',_'CABUYAO') >= 4
Constraint Demand_Constraint_CALAMBA: 56*x_large_CALAMBA + 18*x_small_CALAMBA + 56*y_large_('CALAMBA',_'CABUYAO') + 18*y_small_('CALAMBA',_'CABUYAO') >= 7


In [11]:
# Define the problem
prob2 = pulp.LpProblem("Incoming_Bus_Allocation2", pulp.LpMinimize)

In [12]:
# Decision Variables
x_small = pulp.LpVariable.dicts("x_small", solo_routes, lowBound=0, cat='Integer')
x_large = pulp.LpVariable.dicts("x_large", solo_routes, lowBound=0, cat='Integer')
y_small = pulp.LpVariable.dicts("y_small", hybrid_routes, lowBound=0, cat='Integer')
y_large = pulp.LpVariable.dicts("y_large", hybrid_routes, lowBound=0, cat='Integer')

# Objective Function
prob2 += (
    pulp.lpSum([cost_small_bus[i] * x_small[i] for i in solo_routes if cost_small_bus.get(i, 0) > 0]) +
    pulp.lpSum([cost_large_bus[i] * x_large[i] for i in solo_routes if cost_large_bus.get(i, 0) > 0]) +
    pulp.lpSum([cost_small_hybrid_route.get((i, j), 0) * y_small[(i, j)] for (i, j) in hybrid_routes if cost_small_hybrid_route.get((i, j), 0) > 0]) +
    pulp.lpSum([cost_large_hybrid_route.get((i, j), 0) * y_large[(i, j)] for (i, j) in hybrid_routes if cost_large_hybrid_route.get((i, j), 0) > 0])
)

# Constraints
for i in solo_routes:
    prob2 += (
        (capacity_small_bus * x_small[i] if cost_small_bus.get(i, 0) > 0 else 0) +
        (capacity_large_bus * x_large[i] if cost_large_bus.get(i, 0) > 0 else 0) +
        pulp.lpSum([capacity_small_bus * y_small[(i, j)] for j in solo_routes if cost_small_hybrid_route.get((i, j)) is not None]) +
        pulp.lpSum([capacity_large_bus * y_large[(i, j)] for j in solo_routes if cost_large_hybrid_route.get((i, j)) is not None]) #+
        #pulp.lpSum([capacity_small_bus * y_small[(j, i)] for j in solo_routes if cost_small_hybrid_route.get((j, i)) is not None]) +
        #pulp.lpSum([capacity_large_bus * y_large[(j, i)] for j in solo_routes if cost_large_hybrid_route.get((j, i)) is not None])
        >= demand.get(i, 0), f"Demand_Constraint_{i}"
    )

# Solve the problem
prob2.solve()

# Print the results
for v in prob2.variables():
    print(v.name, "=", v.varValue)

print("Total Cost = ", pulp.value(prob2.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/hk/l0w5001d7pdc860p4ssywh3h0000gn/T/7e5af24c2b044599973a65ca15a6cfba-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/hk/l0w5001d7pdc860p4ssywh3h0000gn/T/7e5af24c2b044599973a65ca15a6cfba-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 80 RHS
At line 87 BOUNDS
At line 105 ENDATA
Problem MODEL has 6 rows, 17 columns and 17 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1965.89 - 0.00 seconds
Cgl0003I 0 fixed, 10 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 4 strengthened rows, 0 substitutions
Cgl0004I processed model has 5 rows, 10 columns (10 integer (0 of which binary)) and 1

In [13]:
for name, constraint in prob2.constraints.items():
    print(f"Constraint {name}: {constraint}")

Constraint Demand_Constraint_ALABANG: 56*x_large_ALABANG + 18*x_small_ALABANG + 56*y_large_('ALABANG',_'CARMONA') + 18*y_small_('ALABANG',_'CARMONA') >= 4
Constraint Demand_Constraint_BINAN: 56*x_large_BINAN + 18*x_small_BINAN + 56*y_large_('BINAN',_'CARMONA') + 18*y_small_('BINAN',_'CARMONA') >= 6
Constraint Demand_Constraint_CARMONA: 56*x_large_CARMONA >= 4
Constraint Demand_Constraint_BALIBAGO: 56*x_large_BALIBAGO + 18*x_small_BALIBAGO >= 31
Constraint Demand_Constraint_CABUYAO: 56*x_large_CABUYAO + 18*x_small_CABUYAO >= 4
Constraint Demand_Constraint_CALAMBA: 56*x_large_CALAMBA + 18*x_small_CALAMBA + 56*y_large_('CALAMBA',_'CABUYAO') + 18*y_small_('CALAMBA',_'CABUYAO') >= 7
